In [1]:
import pandas as pd
import numpy as np

In [2]:
rawGameData = pd.read_csv("spreadspoke_scores.csv")
rawTeamData = pd.read_csv("nfl_teams.csv")

# gameData and teamData should be cleaned using Sean's procedure

# From Data88_Final_Sean_V1:
recentGameData = rawGameData.loc[rawGameData['schedule_season'] >= 2002]
gameData = recentGameData[['schedule_season', 'schedule_week', 'schedule_playoff', 'team_home', 'score_home', 'team_away', 'score_away']]
# End Sean's cleaning

# Ayush's cleaning (cleaning the teams data set):
recentTeamData = rawTeamData.dropna(subset=["team_division"])
teamData = recentTeamData[["team_name", "team_id", "team_division"]]

# Since the conference data is encapsulated in the divisional data, we can just use this function to get it.
teamData["team_conference"] = teamData["team_division"].apply(lambda x: x[:3])
## -- IMPORTANT NOTE -- ##
# There was a row in teamData where the team division did not match the team conference - the New York Jets (index 29)
#    are listed as being in the AFC East Division but the NFC conference in the original data set. I believe this is a
#    mistake, but the above line takes care of that by making the conference AFC.

# End Ayush's cleaning

<ipython-input-2-772fe72af720>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teamData["team_conference"] = teamData["team_division"].apply(lambda x: x[:3])


In [3]:
gameData

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,team_away,score_away
7872,2002,1,False,New York Giants,13,San Francisco 49ers,16
7873,2002,1,False,Buffalo Bills,31,New York Jets,37
7874,2002,1,False,Carolina Panthers,10,Baltimore Ravens,7
7875,2002,1,False,Chicago Bears,27,Minnesota Vikings,23
7876,2002,1,False,Cincinnati Bengals,6,San Diego Chargers,34
...,...,...,...,...,...,...,...
12673,2019,Division,True,Green Bay Packers,28,Seattle Seahawks,23
12674,2019,Division,True,Kansas City Chiefs,51,Houston Texans,31
12675,2019,Conference,True,Kansas City Chiefs,35,Tennessee Titans,24
12676,2019,Conference,True,San Francisco 49ers,37,Green Bay Packers,20


In [4]:
teamData

,team_name,team_id,team_division,team_conference
0,Arizona Cardinals,ARI,NFC West,NFC
3,Atlanta Falcons,ATL,NFC South,NFC
4,Baltimore Ravens,BAL,AFC North,AFC
5,Buffalo Bills,BUF,AFC East,AFC
6,Carolina Panthers,CAR,NFC South,NFC
7,Chicago Bears,CHI,NFC North,NFC
8,Cincinnati Bengals,CIN,AFC North,AFC
9,Cleveland Browns,CLE,AFC North,AFC
10,Dallas Cowboys,DAL,NFC East,NFC
11,Denver Broncos,DEN,AFC West,AFC


In [5]:
def getGameType(row):
    """
    A function to determine the type of game that was played.
    If the two teams' divisions match, the game is a divisional one.
    If not, but the two teams' conferences match, the game is a conference one.
    Otherwise, the game is an out-of-conference one.
    """
    
    homeDivision = row["team_home_division"]
    awayDivision = row["team_away_division"]
    if (homeDivision == awayDivision):
        return "Divisional"
    
    homeConference = row["team_home_conference"]
    awayConference = row["team_away_conference"]
    if (homeConference == awayConference):
        return "Conference"
    
    return "Out-of-conference"

In [6]:
# Make columns: "team_home_division", "team_home_conference", "team_away_division", "team_away_conference", and "gameType"
# Create copies of teamData to be used as the home team and away team's data
homeTeamData = teamData.copy(deep=True)
awayTeamData = teamData.copy(deep=True)

# Rename columns "team_home_..." and "team_away_..."
homeTeamData = homeTeamData.rename(mapper=(lambda x: x[0:5] + "home_" + x[5:]), axis="columns")
awayTeamData = awayTeamData.rename(mapper=(lambda x: x[0:5] + "away_" + x[5:]), axis="columns")

# Merge everything "team_home" and "team_name"
withHome = gameData.merge(homeTeamData, left_on="team_home", right_on="team_home_name")
fullData = withHome.merge(awayTeamData, left_on="team_away", right_on="team_away_name")

# Apply the getGameType function to each row and create the new gameType column
fullData["gameType"] = fullData.apply(lambda singleRow: getGameType(singleRow), axis=1)
fullData

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,team_away,score_away,team_home_name,team_home_id,team_home_division,team_home_conference,team_away_name,team_away_id,team_away_division,team_away_conference,gameType
0,2002,1,False,New York Giants,13,San Francisco 49ers,16,New York Giants,NYG,NFC East,NFC,San Francisco 49ers,SF,NFC West,NFC,Conference
1,2007,7,False,New York Giants,33,San Francisco 49ers,15,New York Giants,NYG,NFC East,NFC,San Francisco 49ers,SF,NFC West,NFC,Conference
2,2008,7,False,New York Giants,29,San Francisco 49ers,17,New York Giants,NYG,NFC East,NFC,San Francisco 49ers,SF,NFC West,NFC,Conference
3,2014,11,False,New York Giants,10,San Francisco 49ers,16,New York Giants,NYG,NFC East,NFC,San Francisco 49ers,SF,NFC West,NFC,Conference
4,2015,5,False,New York Giants,30,San Francisco 49ers,27,New York Giants,NYG,NFC East,NFC,San Francisco 49ers,SF,NFC West,NFC,Conference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4574,2016,16,False,Philadelphia Eagles,24,New York Giants,19,Philadelphia Eagles,PHI,NFC East,NFC,New York Giants,NYG,NFC East,NFC,Divisional
4575,2017,3,False,Philadelphia Eagles,27,New York Giants,24,Philadelphia Eagles,PHI,NFC East,NFC,New York Giants,NYG,NFC East,NFC,Divisional
4576,2018,12,False,Philadelphia Eagles,25,New York Giants,22,Philadelphia Eagles,PHI,NFC East,NFC,New York Giants,NYG,NFC East,NFC,Divisional
4577,2019,14,False,Philadelphia Eagles,23,New York Giants,17,Philadelphia Eagles,PHI,NFC East,NFC,New York Giants,NYG,NFC East,NFC,Divisional


# Win Percentages

In [7]:
# Make a table with five columns: team name, valid, schedule season, divisional win percentage
#    conference win percentage, and out-of-conference win percentage, each team having one row/year.

# Let's create a function that iterates through the entire table and increments values one by one.
# The best method may be to just double the entire table, swapping teams 1 and 2 to ensure that games aren't undercounted.

# The following doubles the table.
def renamer(toProcess, toRecognize, replaceValue):
    result = toProcess.find(toRecognize)
    if result != -1:
        return toProcess[:result] + replaceValue + toProcess[result + len(toRecognize):]
    else:
        return toProcess
    
doubled = fullData.copy()
doubled = doubled.rename(mapper=(lambda value: renamer(value, "home", "1")), axis="columns")
doubled = doubled.rename(mapper=(lambda value: renamer(value, "away", "2")), axis="columns")

doubledCopy = fullData.copy()
doubledCopy = doubledCopy.rename(mapper=(lambda value: renamer(value, "home", "2")), axis="columns")
doubledCopy = doubledCopy.rename(mapper=(lambda value: renamer(value, "away", "1")), axis="columns")

doubled = pd.concat([doubled, doubledCopy], sort=False)
# At this point, all games are double counted.
# To make grouping and later calculations easier, let's add more columns - divisionWin, divisionGame,
#   conferenceWin, conferenceGame, outOfConferenceWin, and outOfConferenceGame.

def gameCalculator(singleRow, desiredGameType):
    return (singleRow["gameType"] == desiredGameType)

def winCalculator(singleRow, desiredGameType):
    return (singleRow["gameType"] == desiredGameType) and (singleRow["score_1"] > singleRow["score_2"])

doubled["divisionWin"] = doubled.apply((lambda singleRow: winCalculator(singleRow, "Divisional")), axis=1)
doubled["divisionGame"] = doubled.apply((lambda singleRow: gameCalculator(singleRow, "Divisional")), axis=1)
doubled["conferenceWin"] = doubled.apply((lambda singleRow: winCalculator(singleRow, "Conference")), axis=1)
doubled["conferenceGame"] = doubled.apply((lambda singleRow: gameCalculator(singleRow, "Conference")), axis=1)
doubled["outOfConferenceWin"] = doubled.apply((lambda singleRow: winCalculator(singleRow, "Out-of-conference")), axis=1)
doubled["outOfConferenceGame"] = doubled.apply((lambda singleRow: gameCalculator(singleRow, "Out-of-conference")), axis=1)

doubled

,schedule_season,schedule_week,schedule_playoff,team_1,score_1,team_2,score_2,team_1_name,team_1_id,team_1_division,...,team_2_id,team_2_division,team_2_conference,gameType,divisionWin,divisionGame,conferenceWin,conferenceGame,outOfConferenceWin,outOfConferenceGame
0,2002,1,False,New York Giants,13,San Francisco 49ers,16,New York Giants,NYG,NFC East,...,SF,NFC West,NFC,Conference,False,False,False,True,False,False
1,2007,7,False,New York Giants,33,San Francisco 49ers,15,New York Giants,NYG,NFC East,...,SF,NFC West,NFC,Conference,False,False,True,True,False,False
2,2008,7,False,New York Giants,29,San Francisco 49ers,17,New York Giants,NYG,NFC East,...,SF,NFC West,NFC,Conference,False,False,True,True,False,False
3,2014,11,False,New York Giants,10,San Francisco 49ers,16,New York Giants,NYG,NFC East,...,SF,NFC West,NFC,Conference,False,False,False,True,False,False
4,2015,5,False,New York Giants,30,San Francisco 49ers,27,New York Giants,NYG,NFC East,...,SF,NFC West,NFC,Conference,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4574,2016,16,False,New York Giants,19,Philadelphia Eagles,24,New York Giants,NYG,NFC East,...,PHI,NFC East,NFC,Divisional,False,True,False,False,False,False
4575,2017,3,False,New York Giants,24,Philadelphia Eagles,27,New York Giants,NYG,NFC East,...,PHI,NFC East,NFC,Divisional,False,True,False,False,False,False
4576,2018,12,False,New York Giants,22,Philadelphia Eagles,25,New York Giants,NYG,NFC East,...,PHI,NFC East,NFC,Divisional,False,True,False,False,False,False
4577,2019,14,False,New York Giants,17,Philadelphia Eagles,23,New York Giants,NYG,NFC East,...,PHI,NFC East,NFC,Divisional,False,True,False,False,False,False


In [8]:
# Now, we can group by the year and team_1 to get stats.
doubled = doubled.rename(columns = {"team_1": "teamName", "schedule_playoff": "valid"})
doubledGrouped = doubled.groupby(["schedule_season", "teamName"], as_index=asIndex)

aggDict = {}
for testColumn in ["divisionWin", "divisionGame", "conferenceWin", "conferenceGame", "outOfConferenceWin", "outOfConferenceGame"]:
    aggDict[testColumn] = np.count_nonzero

aggDict["valid"] = lambda toCount: np.count_nonzero(toCount) >= 1

NameError: name 'asIndex' is not defined

In [ ]:
percentageTable = doubledGrouped.agg(aggDict)
percentageTable

# Finally, calculate percentages.
# Make a table with five columns: team name, valid, schedule season, divisional win percentage
#    conference win percentage, and out-of-conference win percentage, each team having one row/year.

for element in ["division", "conference", "outOfConference"]:
    percentageTable[element + "WinPercent"] = 100 * percentageTable[element + "Win"] / percentageTable[element + "Game"]
    # Uncomment this line for output without number of wins and games in each category.
    # percentageTable = percentageTable.drop(columns=[element + "Win", element + "Game"])
percentageTable

The above table has data for all teams for every year, detailing their win percentages in division, in conference, and out-of-conference. It also has a table named "valid" to specify whether or not they played any playoff games.

In [ ]:
#Hypothesis Testing:

#Null: The plots of a team's winning percentage in division, in conference, and out of conference with that team's playoff 
#      winning percentage that season will have no difference in correlation, r, values.
#Alternative: The plots of a team's winning percentage in division, in conference, and out of conference with that team's playoff 
#             winning percentage that season will have a difference in correlation, r, values.

#Test Statistic: Correlation, r, for a team's relationship between their playoff winning percentage with their in division, in 
#                in conference, and out of conference winning percentages.

In [ ]:
def standard_units(series): #converts a series to be in standard units so we can calculate correlation
    return (series-np.mean(series))/np.std(series)

def r_calculator(x, y): #Will be used to determine the correlation, r
    return np.mean(standard_units(x)*standard_units(y))
    
    